## 음성인식 : Feature representation (Speech_Recognition : Feature representation)
---
__참고사항 :__ 
이 글은 성원용 교수님의 "음성인식 및 합성" 강의와 [Deep Learning for Speech Recognition(Adam Coates, Baidu)](https://www.youtube.com/watch?v=g-sndkf7mCs)를 주로 참고하여 정리하였습니다.  
정리글이므로 오류가 있을 수 있으니 github issue 란에 제보해주시면 확인 후 수정하겠습니다.  
감사합니다.
<br \>

### 1. 음성의 원리
사람의 음성에는 말의 내용을 담고 있는 "말소리"와 "음"으로 되어 있다.  
"말소리"는 성도(vocal track)의 움직임으로 결정되고, "음"은 성대(vocal cord)의 모양과 성도의 길이로 결정된다.  
성도의 길이가 길면 낮은 음이, 짧으면 높은 음이 난다.

#### 성도(Vocal Track), 성대(Vocal Folds) 
![vocal track and vocal cord](./img/vocal_track.jpg "vocal track and vocal cord")
출처: [englicious.org](http://www.englicious.org/lesson/phonetics-and-phonology-introduction/phonetics-and-phonology-consonants)  
<br \>
성도를 이루고 있는 혀(Tongue), 입술(Lips) 등등이 모양을 바꾸면서 다른 말소리를 내게 된다.  
동영상 [Utterance robot finally got the same voice as humans](https://www.youtube.com/watch?v=HmSYnOvEueo)의 2:14 부분을 보면 성도가 바뀜에 따라 목소리가 달라짐을 기계를 통해 확인할 수 있다.  
영상에서 기계는 "마미므메~"와 같이 성도를 움직여가며 다른 말소리를 내지만 음의 높낮이는 변하지 않는다.  
그 이유는 성대역할을 하는 부분이 변형이 없고, 성도의 길이도 일정하기 때문일 것이다.  
참고 : [목소리의 발성원리]("http://soryro.tistory.com/71")  
<br \>  
음성 wave에서 말소리는 파형으로 나타나고, 음은 주파수로 나타난다.  
다른 말소리는 다른 파형을 갖게 된다(아래 예시에서 사용된 파형은 임의로 그린 것이다).
![empty](./img/different_waveform.png "different_waveform")  
같은 말소리라도 높은 주파수에서는 높은 음으로, 낮은 주파수에서는 낮은 음으로 나타난다.  
목소리에서 음을 pitch라고 한다.
![empty](./img/different_pitch.png "different_pitch")  
<br \>  
사실 기본적인 음성인식을 하는데에 pitch는 불필요한 정보이다.  
저음의 남자가 "나는 학교에 간다"라고 말한 것과 고음의 여자가 "나는 학교에 간다"라고 말한 것 중 어느 것을 음성인식하더라도 음성인식의 결과는 똑같아야 하기 때문이다. 이런 불필요한 pitch 정보가 음성에 포함되면 필요한 연산량을 증대시키게 된다. 그래서 전통적 음성인식에서는 MFCC(Mel Frequency Cepstral Coefficients)를 사용해 음성에서 pitch 정보를 없애게 된다. MFCC를 거친 음성은 중성적인 목소리로 변하며, 이는 원본 정보가 어느 정도 손실됨을 의미한다. 

현대의 음성인식에서는 승산기(multiflier) 가격 하락과 분산 처리 시스템의 발달로 연산 가격이 하락함에 따라 MFCC에서 pitch 정보가 소실되기 전 단계인 Mel scaled filter bank 까지만 거친다. 이를 통해 말하는 사람까지 구분하는 음성인식이 가능해진다.  
<br \>
MFCC에 대한 설명 이전에 샘플링과 푸리에 변환에 대한 정리를 먼저 한다.  
<br \>  

### 2. 샘플링 (Sampling)
음성 wave는 아날로그 신호(Analog signal)로 연속적(continuous)이다.  
컴퓨터는 이산 데이터(discrete data)만 처리할 수 있으므로, 아날로그 신호를 샘플링(Sampling)을 통해 이산적(discrete)으로 바꾼 디지털 신호(Digital signal)로 만들어야 한다.  

신호 분석이 천천히 이뤄져도 되면 샘플링 주파수는 낮아도 된다.  
그러나 신호 분석이 빨리 이뤄져야 하면 샘플링 주파수는 높아야 한다.  
컴퓨터 입장에서는 샘플링 주파수가 높을수록 처리할 데이터가 많아지게 된다.  

따라서 컴퓨터는 샘플링 주파수가 낮을수록 좋지만, 샘플링 주파수를 마냥 낮출수는 없다. Aliasing problem이 존재하기 때문이다.

- __Aliasing problem__  
샘플링 주파수가 너무 낮아서, 디지털 신호를 보면 원래 신호보다 낮은 주파수로 보이는 현상이다.
![empty](./img/aliasing.png "aliasing")
출처 : 성원용 교수님 슬라이드 
<br \>  
1Hz로 샘플링 했을 때, 주파수 0.1Hz, 1.1Hz, 2.1Hz 신호들이 모두 같은 주파수로 보이는 것을 볼 수 있다.  

Aliasing problem을 막기 위해서는 Nyquist sampling theorem에 따라 샘플링을 해야한다.  
- __Nyquist sampling theorem__  
어떤 입력 신호의 최대 주파수가 fmax이면 fsampling(샘플링 주파수) > 2*fmax를 만족해야 한다는 이론이다.  
<br \>  

### 3. 주파수 영역 표현 (Frequency domain representation)
음성 wave(Time domain representation)을 주파수 영역 표현(Frequency domain representation)으로 바꾸면 음성인식에서 편리한 점이 많다.  
예를 들어 확보한 음성샘플에 버스 경적소리가 포함된 경우를 생각해보자. 연구자는 음성인식에 방해가 되는 버스 경적소리를 제거하고 싶을 것이다.  
이때 음성 wave를 주파수 영역 표현으로 바꿔 버스 경적소리와 실험 대상자의 음성 주파수를 분석한다면, 버스 경적 주파수에 해당되는 소리를 제거할 수 있다.  

그렇다면 음성 wave를 어떻게 주파수 영역 표현으로 바꿀까?  
푸리에 변환(Fourier transform)을 사용하면 된다.  
아래 동영상은 푸리에 변환을 직관으로 이해하는데 매우 도움이되어서 링크를 첨부한다.  
참고 : [푸리에 변환이 대체 뭘까요? 그려서 보여드리겠습니다.](https://www.youtube.com/watch?v=spUNpyF58BY)  

- __푸리에 변환__  
과학자 푸리에는 복잡한 파형의 wave는 사실 다양한 주파수의 간단한 wave들의 합으로 표현될 수 있다는 것을 알아내었다.
![empty](./img/different_sine_wave.png "different_sine_wave")
출처 : [푸리에 변환이 대체 뭘까요? 그려서 보여드리겠습니다.](https://www.youtube.com/watch?v=spUNpyF58BY)  
<br \>
그러면 간단한 wave를 주파수 기반으로 표현하면 좋을 텐데, 그 방법이 푸리에 변환인 것이다.  
주파수는 1초 동안 반복되는 주기 수이다. 1초를 평면 상 원 한 바뀌를 도는 것이라고 생각하면, 아래와 같이 표현될 수 있다.  
음성 wave에서의 진폭은 평면 상 원에서는 원점으로부터 떨어진 vector의 크기가 된다.  
![empty](./img/frequency_at_2d.png "frequency_at_2d")
출처 : [푸리에 변환이 대체 뭘까요? 그려서 보여드리겠습니다.](https://www.youtube.com/watch?v=spUNpyF58BY)  
<br \>
이때 주파수를 조금씩 바꿔가며 무게중심을 구한 뒤, 무게중심의 x좌표만 살펴보면 대부분이 0에 가깝다가 갑자기 양의 값을 갖는 구간이 등장한다.  
(아래 그림은 원래 wave에서 음수를 허용하여 아래쪽으로 이동시킨 것이다. 원래 wave 그대로 무게중심의 x 좌표들을 구한다면 처음 구간에는 양의 값을 갖는다.)  
이 값이 거의 푸리에 변환으로 도출한 해당 wave의 주파수인 것이다.
![empty](./img/peak.png "peak")
출처 : [푸리에 변환이 대체 뭘까요? 그려서 보여드리겠습니다.](https://www.youtube.com/watch?v=spUNpyF58BY)  
<br \>
거의 푸리에 변환을 통해 복잡한 파형의 wave도 아주 간단하게 주파수 기반으로 표현할 수 있다.  
만약 2Hz가 실험 대상자의 음성 주파수이고 3Hz가 버스경적 주파수라면, 연구자는 3Hz 주파수를 low pass filter로 제거해버릴 수 있을 것이다.
![empty](./img/almost_fourier_transform.png "almost_fourier_transform")
출처 : [푸리에 변환이 대체 뭘까요? 그려서 보여드리겠습니다.](https://www.youtube.com/watch?v=spUNpyF58BY)  
<br \>
지금의 방법을 거의 푸리에 변환이라고 표현하는 이유는 실제 푸리에 변환식은 무게중심의 x 좌표만 구한 것과 약간의 차이가 있기 때문이다.  
2차원 평면의 점을 두 개의 좌표가 아닌 하나의 좌표로 표현하기위해 복소평면의 한 점으로 표현한다.  
그래서 복소수와 자연상수 e의 지수 표현이 들어가게되며, 무게중심과 달리 $\left( \frac{1}{(t_2-t_1)} \right)$을 곱해주지 않는다.  
결론적으로 푸리에 변환은 아래와 같은 식이 된다.  
<br \>
$$ \hat{g}(f) = \int_{t_1}^{t_2}g(t)e^{-j2\pi ft}\, dt$$  
<br \>
우리가 음성인식을 할 때에 아날로그 신호(Analog signal)인 음성을 샘플링을 통해 디지털 신호(Digital signal)로 바꿔 사용한다.  
그래서 실제로는 위에 적힌 연속 시간 신호 푸리에 변환(Continuous Time Fourier Transform, CTFT)이 아닌 이산 푸리에 변환(Discrete Fourier Transform, DFT)을 사용한다(CTFT는 시간 영역에도 continuous하고 주파수 영역에서도 continuous하지만, DFT는 시간 영역에서도 discrete하고, 주파수 영역에서도 discrete하다).  
<br \>
$$ X[k] = \sum_{n=0}^{N-1}x[n]e^{-j2\frac{\pi}{N} kn}$$  
참고 : [위키백과 이산 푸리에 변환](https://ko.wikipedia.org/wiki/%EC%9D%B4%EC%82%B0_%ED%91%B8%EB%A6%AC%EC%97%90_%EB%B3%80%ED%99%98),  [DTFT, DTF](http://bluelimn.tistory.com/entry/%EC%9D%B4%EC%82%B0-%EC%8B%9C%EA%B0%84-%EC%8B%A0%ED%98%B8%EC%9D%98-%ED%91%B8%EB%A6%AC%EC%97%90-%EB%B3%80%ED%99%98)  
<br \>  

### 4. 디지털 필터  (Digital filter)
위의 버스 경적소리 예시처럼 특정 주파수를 제거하고 싶은 경우가 있다. 이때 사용하는 것이 필터이다.  
필터의 종류는 Low pass filter, High pass filter, Band pass filter가 있다.  
Low pass filter는 낮은 주파수만 통과시키고, High pass filter는 높은 주파수만, 그리고 Band pass는 지정한 주파수 범위만 통과시킨다.  
이상적인 필터의 모습은 아래와 같다.  
![empty](./img/filter.jpg "filter")
출처 : ELEC 202 Circuit Analysis II  

- __Low pass filter__  
간단한 Low pass filter는 과거 신호의 이동 평균으로 나타낼 수 있다. 과거 신호들을 이동 평균한 선을 보면 주파수가 더 낮아진 것을 볼 수 있다.  
주식 차트에서의 이동 평균선의 예시가 가장 직관적으로 쉽게 이해되었다.
![empty](./img/moving_average.png "moving_average")
출처 : [치킨요정의 공부방](http://md2biz.tistory.com/110)   
<br \>
그림에서 5일, 10일, 20일, 60일, 120일 총 5개의 이동평균선이 그려져있다.  
빨간선인 5일 이동 평균선이 회색선인 120일 이동 평균선에 비해 훨씬 더 구불구불하니 주파수가 높은 것을 볼 수 있다.  
<br \>
일반적인 구조의 FIR filter (Finite Impulse Response filter)가 이와 비슷하다.  
<br \>
$$ \begin{matrix}
    y[n] = \sum h[k]x[n-k] {\scriptstyle\text { for k = 0 to M-1}}  \\
    \text{where h[k] is an array of constants (filter coeifficients)}
    \end{matrix} $$  
![empty](./img/FIR_filter.png "FIR_filter")
출처 : [Lab Book Pages](http://www.labbookpages.co.uk/audio/firWindowing.html)  
<br \>
결과물은 이상적인 필터와 달리 아래의 모양과 유사하다.  
![empty](./img/low_pass_filter.png "low pass filter")
출처 : [http://msp.ucsd.edu](http://msp.ucsd.edu/techniques/v0.11/book-html/node129.html)  
<br \>
ripple과 stopband가 최대한 평평하고, transition band가 좁을수록 이상적인 필터에 가까운데, 이를 위해서는 필터의 차수를 늘려야한다.  
필터의 차수를 늘린다는 이야기는 이동 평균선을 구할 때 이용되는 날짜 수를 늘리는 것과 일맥상통하다.  
이렇게 Low pass filter를 구했다면, High pass filter와 Band pass filter는 구하기 수월해진다.  
<br \>
- __High pass filter and Band pass filter__  
High pass filter는 All pass filter - Low pass filter로 구할 수 있다.  
![empty](./img/high_pass_filter.png "high pass filter")
출처 : [Lab Book Pages](http://www.labbookpages.co.uk/audio/firWindowing.html)  
<br \>
Band pass filter는 Low pass filter와 High pass filter를 적절히 조합해서 만들 수 있다.  
자세한 사항은 [Lab Book Pages](http://www.labbookpages.co.uk/audio/firWindowing.html)  을 참고하면 좋을 것 같다.  

FIR filter 이외에도 IIR filter, Adaptive fitler 등이 있으나 정리는 FIR까지만 ^~^;